In [1]:
import requests, scrapy
from scrapy.http import TextResponse

## checklist

### 진행예정
 1) 모듈들 import 하는 건 패키지 만들 때 스타트업 파일처럼 걸리게 해보자 \
 2) 추가필요 열: 플랫폼명, 날짜 \
 3) 타 플랫폼과 합쳤을 때 형식 통일하도록 data type과 Nan 처리방법 등 정의 \
 4) 룸 이름에 싱글, 더블 여부 없이 스탠다드라고만 되어있는데, 침대 정보도 열추가해야할 거 같은데?
 

### 완료
 1) page바꿔가면서 했을 때 상품들 연속성 CHK (limit=50에 따른 상품 누락 여부 확인) -> 이상없음. \
 2) page=1, 2 이상 구분할 때 [4], [1]이 아니라 '전체'를 가져오게 해야겠다. 시간 저녁되니까 숫자로 안된다. -> 해결 \
 3) today 기준시 기준으로, 한국보다 9시간 느림 -> 이거 맞춰줘야 함 -> 해결
 4) 리뷰 추가 필요 -> 해결

### 추후 체크 디버깅 예상
 1) score, roomtype, price 중 현재 데일리호텔은 score이슈만 발생. 나머지도 추후 체크 필요 예상. \
 2) headers에서 app-version 나중에 업데이트 필요할 수도 있음. 데이터 들어오는데 숫자 앉맞으면 확인필요 예상되는 인자.

In [2]:
# 예상 프로세스
# 1) 첫째 페이지에서 호텔리스트별 hotelId파악하여 링크주소 모두 확보, 그리고 각 링크에 접속해서 이름, 객실 정보 등 크롤링한 후 리스트에 append
# 2) 두번째 페이지부터 호텔리스트별 hotelId파악하여 링크주소 모두 확보, 그리고 각 링크에 접속해서 이름, 객실 정보 등 크롤링한 후 리스트에 append
# 3) 페이지수 1씩 늘리면서 에러날 때까지 돌아가는 루프 작성
# 4) 루프 성공적으로 다 돌아간다면, 리스트들로 데이터프레임화하는 거까지
# 5) 날짜 등 인수로 받아서 데이터프레임 반환하는 함수 만들기

# 함수화

In [1]:
import scrapy, requests
from scrapy.http import TextResponse
import datetime


KST = datetime.timezone(datetime.timedelta(hours=9))
today = datetime.datetime.now(tz=KST)
start_date= today.strftime("%Y-%m-%d") # 시작일 디폴트 = 오늘
end_date = (today + datetime.timedelta(days=1)).strftime("%Y-%m-%d") # 종료일 디폴트 = 내일
staydays = (datetime.datetime.strptime(end_date,'%Y-%m-%d').date() - \
            datetime.datetime.strptime(start_date,'%Y-%m-%d').date()).days # 숙박일수 디폴트 = 1박start_date, end_date


def dailyhotel(start_date= today.strftime("%Y-%m-%d"),\
               end_date = (today + datetime.timedelta(days=1)).strftime("%Y-%m-%d"),\
               persons=2):
    
    
    # 사전 import & settings
    import scrapy, requests
    from scrapy.http import TextResponse
    import datetime

    today = datetime.date.today()
    staydays = (datetime.datetime.strptime(end_date,'%Y-%m-%d').date() - \
                datetime.datetime.strptime(start_date,'%Y-%m-%d').date()).days

    
    # 계층 관계에 의해 지워지지 않고 모두 누적되어야 하는 리스트 최상단 배치
    names, levels, scores, locations, roomtypes, prices, reviews, hotel_desc_urls = [], [], [], [], [], [], [], []


    # 호텔리스트 가져올 parent_url 생성
    page = 1
    while True:
        parent_url = f"https://www.dailyhotel.com/newdelhi/goodnight/api/v4/stays/sales?dateCheckIn={start_date}&regionIdx=30070&shortCutType=hotel_resort&persons={persons}&stays={staydays}&details=true&regionStayType=hotel_resort&stayType=hotel,resort&page={page}&limit=50"
        req = requests.get(parent_url)
        response = req.json()
        list_all = response['data']['staySaleSections']

        if len(list_all) > 0:
            for each in list_all:
                if each['sectionViewType'] == 'NORMAL_섹션뷰타입':
                    lists = each['staySales']
                    print('page=', str(page))
                    page += 1
        else:
            print('All registered hotels have been searched.') # 이거는 확인용
            break


        # parent_url에서 각 호텔들의 Index number 가져오기
        hotelidx = []
        num = 1
        for each in lists:
            hotelidx.append(each['hotelIdx'])


        # 가져온 index number 사용, 각 호텔들의 정보들을 각 해당 리스트(names, levels...)에 누적
        for idx in hotelidx:
            hotel_url = f'https://www.dailyhotel.com/newdelhi/goodnight/api/v9/hotel/{idx}?stays={staydays}&dateCheckIn={start_date}&regionStayType=all'
            headers = {
                'app-version': '2.2.13'
            }
            req = requests.get(hotel_url, headers=headers)
            response = TextResponse(req.url, body=req.text, encoding='utf-8')
            resp_content = response.json()


            name = resp_content['data']['name'] 
            level = resp_content['data']['stayGradeName'] 
            try:
                score = round(resp_content['data']['rating']['values']/100*5, 1) 
            except:
                score = '점수없음'
            location = resp_content['data']['address']
            roomtype = []
            for each in resp_content['data']['rooms']:
                roomtype.append(each['roomName'])
            price = []
            for each in resp_content['data']['rooms']:
                price.append(each['amount']['discountAverage'])
            review = resp_content['data']['statistic']['reviewScoreTotalCount']
            hotel_desc_url = f"https://www.dailyhotel.com/stays/{idx}?dateCheckIn={start_date}&stays={staydays}" # 크롤링이랑 예약링크 다름 -> request url과 다르도록!

            
            name = [name] * len(roomtype)
            level = [level] * len(roomtype)
            score = [score] * len(roomtype)
            location = [location] * len(roomtype)
            review = [review] * len(roomtype)
            hotel_desc_url = [hotel_desc_url] * len(roomtype)
            if len(roomtype) == len(price):
                print(str(num), ':', name[0])
                num += 1
            else:
                raise Exception('개수오류')

                
        # 데이터 추가
            names.extend(name)
            levels.extend(level)
            scores.extend(score)
            locations.extend(location)
            roomtypes.extend(roomtype)
            prices.extend(price)
            reviews.extend(review)
            hotel_desc_urls.extend(hotel_desc_url)
            
            
    # 데이터프레임화
    df = pd.DataFrame({
        'Name': names,
        'Level': levels,
        'Score': scores,
        'Review': reviews,
        'Location': locations,
        'Roomtype': roomtypes,
        'Price': prices,
        'Link': hotel_desc_urls,
    })
    
    
    return df

In [2]:
test = dailyhotel()
print(test.shape)
test.tail()

page= 1
1 : 노보텔 앰배서더 강남
2 : 몬드리안 서울 이태원(Mondrian Seoul Itaewon)
3 : 글래드 여의도
4 : 명동 퍼시픽 호텔
5 : ENA 스위트 호텔 남대문
6 : 이비스 앰배서더 명동
7 : 카푸치노 호텔
8 : 그랜드 머큐어 앰배서더 호텔 앤 레지던스 서울 용산
9 : 노보텔 스위트 앰배서더 서울 용산
10 : 카라쉬 호텔 서울
11 : 호텔 그레이스리 서울
12 : 서머셋 팰리스
13 : 라마다 서울 동대문
14 : 라마다 호텔앤스위트 서울 남대문
15 : 그랜드 인터컨티넨탈 서울 파르나스
16 : 인터컨티넨탈 서울 코엑스
17 : 글래드 마포
18 : 신라스테이 마포
19 : 코트야드 메리어트 서울 보타닉 파크
20 : 스위스 그랜드 호텔
21 : 코트야드 바이 메리어트 서울 타임스퀘어
22 : 나인트리 호텔 동대문
23 : L7 명동 바이 롯데
24 : 나인트리 프리미어 호텔 인사동
25 : 포포인츠 바이 쉐라톤 명동
26 : 서울신라호텔
27 : 서울가든호텔
page= 2
1 : 나인트리 프리미어 명동2
2 : L7 강남 바이 롯데
3 : 롯데시티호텔 명동
4 : 솔라리아 니시테츠 명동
5 : 신라스테이 서대문
6 : 호텔 뉴브
7 : 포포인츠 바이 쉐라톤 서울 구로
8 : 신라스테이 구로
9 : 신라스테이 광화문
10 : 노보텔 앰배서더 동대문 호텔 & 레지던스
11 : 임피리얼 팰리스 서울
12 : 오라카이 대학로 호텔
13 : L7 홍대 바이 롯데
14 : 콘래드 서울
15 : 노보텔 앰배서더 독산
16 : 신라스테이 서초
17 : 포시즌스 호텔 서울
18 : 그랜드 하얏트 서울
19 : 아만티 호텔 서울 홍대
20 : 트레블로지 명동 을지로 호텔
21 : 롯데 시티호텔 김포공항
22 : SR 호텔 서울
23 : 코리아나 호텔
24 : 홀리데이인 익스프레스 서울 홍대
25 : 롯데 호텔 월드
26 : 호텔 피제이 명동
27 : 알로프트 서울 강남
28 : 홍대 나인브릭 호텔
29 : 글래드 강남 코엑스센터


,Name,Level,Score,Review,Location,Roomtype,Price,Link
1721,합정 써클 호텔,디자인,3.8,114,서울 마포구 양화로 15,[23시체크인] 스탠다드,100000.0,https://www.dailyhotel.com/stays/28977?dateChe...
1722,합정 써클 호텔,디자인,3.8,114,서울 마포구 양화로 15,디럭스,110000.0,https://www.dailyhotel.com/stays/28977?dateChe...
1723,합정 써클 호텔,디자인,3.8,114,서울 마포구 양화로 15,[23시체크인] 디럭스,110000.0,https://www.dailyhotel.com/stays/28977?dateChe...
1724,합정 써클 호텔,디자인,3.8,114,서울 마포구 양화로 15,스위트,120000.0,https://www.dailyhotel.com/stays/28977?dateChe...
1725,합정 써클 호텔,디자인,3.8,114,서울 마포구 양화로 15,[23시체크인] 스위트,120000.0,https://www.dailyhotel.com/stays/28977?dateChe...
